In [1]:
from modelEq import saturation, alloAct, alloInhib
from util import printBlockMessage
from units import cnt2mol
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from random import randint
from scipy.constants import Avogadro
from plotly.subplots import make_subplots
from dataclasses import dataclass

DICT_TIME = {
    'simDurationSeconds':5000, #172800, #48 hrs if 1s
    'simStepsPerSecond': 1,
    }
# Format: moleculename: (concentration uM, type) 
# Type can be 'met' for metabolite or 'enz' for enzyme
DICT_T0_CONC = {
            # Changeable met concs
            'Gluc' : (5000, 'met'),
            'F6P' :  (0, 'met'),          # Goal: 8.8mM - https://www.nature.com/articles/nchembio.186.pdf,
            'F16BP' :(0, 'met'),        # Goal: 15.0mM - https://www.nature.com/articles/nchembio.186.pdf
            'PEP' :  (0, 'met'),         # Goal: 0.18mM - https://www.nature.com/articles/nchembio.186.pdf
            'ADP' :  (560, 'met'),           # Goal: 0.56mM - https://www.nature.com/articles/nchembio.186.pdf
            'ATP' :  (9600, 'met'),
            'PYR' :  (0, 'met'),
            # Changeable enz concs
            'PFKa' : (1e-3, 'enz'),
            'PFKi' : (1e-3, 'enz'),
            'PFKb' : (5e-3, 'enz'),
            # Unchanging Enzyme Concs:
            'HK' :   (7e-3, 'enz'),
            'PEPWELD' : (4.5e-3, 'enz'),
            'PK' :   (7.0e-2, 'enz'),
        }
DICT_KCAT = {
    'PFKa':1e2,
    'PFKi':1e2,
    'HK' : 4e3,
    'PK' : 1.7e3,
    'PEPWELD' : 1e3
    }
DICT_K = {
    'PFKa-->PFKb':1e-3,
    'PFKi-->PFKb':1e-3,
    'PFKa-->PFKi':cnt2mol(5e11, 'micro'),
    'PFKi-->PFKa':cnt2mol(5e11, 'micro'),
    }
DICT_KM = {
    'PFKb_PFKb':5e-7,
    'PFKb_PFKa':1e-2,
    'PFKb_PFKi':1e-2,
    'PFKb_ADP': 550.0,
    'PFKb_PEP': 5e-2,
    'PFKa_F6P': 1.0,
    'PFKa_ATP': 60,
    'PFKi_F6P': 1e7,
    'PFKi_ATP': 60,
    'HK_Gluc' : 500.0,
    'HK_ATP' : 15000.0,
    'PEPWELD_F16BP' : 5000.0,
    'PK_PEP' : 35.0,
    'PK_ADP' : 50.0,
    }
DICT_K_REGULATORY = {
    # Format: Ka/Ki_Molecule_Target
    # Ki
    'Ki_PYR_HK' : 50.0,
    'Ki_PEP_PFK' : 100.0,
    'Ki_ATP_PK' : 12000.0,
    'Ki_PEP_PEPWELD' : 400.0,
    'Ki_F6P_HK' : 20.0,
    # Ka
    'Ka_F16BP_PK' : 100.0
}

DICT_ENZ_SUBSTRATE = {
    'HK' : {'subs':[('Gluc','sat'), ('ATP','sat')], 'prod':[('F6P','sat'), ('ADP','sat')], 'alloI':[('F6P','AlloI')],},
    'PFKa' :  {'subs':[('F6P','sat'), ('ATP','sat')], 'prod':[('F16BP','sat'), ('ADP','sat')]},
    'PFKi' :{'subs':[('F6P','sat'), ('ATP','sat')], 'prod':[('F16BP','sat'), ('ADP','sat')]},
    'PFKb' : {'subs':[('ADP','sat'), ('PEP','sat')], 'prod':[('PFKa','sat'), ('PFKi','sat')]},
    'PEPWELD' : {'subs':[('F16BP','sat')], 'prod':[('PEP','sat')]},
    'PK' : {'subs':[('PEP','sat'), ('ADP','sat')], 'prod':[('PYR','sat'), ('ATP','sat')], 'alloI':[('ATP', 'AlloI')], 'alloA':[('F16BP', 'AlloA')]},    
}

In [13]:
@dataclass 
class KValue:
    ktype:str = ""
    ktarget:str = ""
    kval:float = 0.0
    
mol = {
        # Changeable met concs
        'Gluc' : {'val':5000.0, 'unit':'uM','moltype':'met', 'kval':[
            KValue("sat", "HK", 500.0)
        ]},
        'F6P' : {'val':0.0, 'unit':'uM','moltype':'met', 'kval':[
            KValue("sat", "PFKa", 1.0),
            KValue("sat", "PFKi", 1e7),
            KValue("alloI", "HK", 20.0),
        ]},
        'F16BP' : {'val':0.0, 'unit':'uM','moltype':'met', 'kval':[
            KValue("sat", "PEPWELD", 5000.0),
            KValue("alloA", "PK", 400.0)
        ]},
        'PEP' : {'val':0.0, 'unit':'uM','moltype':'met', 'kval':[
            KValue("sat", "PK", 35.0),
            KValue("sat", "PFKb", 5e-2),
        ]},
        'ADP' : {'val':560.0, 'unit':'uM','moltype':'met', 'kval':[
            KValue("sat", "PK", 50.0),
            KValue("sat", "PFKb", 550.0),
        ]},
        'ATP' : {'val':9600.0, 'unit':'uM','moltype':'met', 'kval':[
            KValue("sat", "PFKa", 60.0),
            KValue("sat", "PFKi", 60.0),
            KValue("sat", "HK", 15000.0),
            KValue("alloI", "PK", 12000.0),
        ]},
        'PYR' : {'val':0.0, 'unit':'uM','moltype':'met', 'kval':[]},
        # Changeable enz concs
        'PFKa' : {'val':1e-3, 'unit':'uM','moltype':'enz', 'kval':[
            # Note this will change b/c some enzymes can catalyze more than 1 reaction with different kvals
            KValue("kcat", "", 1e2)
        ]},
        'PFKi' : {'val':1e-3, 'unit':'uM','moltype':'enz', 'kval':[
            KValue("kcat", "", 1e2)
        ]},
        'PFKb' : {'val':5e-3, 'unit':'uM','moltype':'enz', 'kval':[
            KValue("kcat", "", 0.0)
        ]},
        # Unchanging Enzyme Concs:
        'HK' : {'val':7e-3, 'unit':'uM','moltype':'enz', 'kval':[
            KValue("kcat", "", 4e3)
        ]},
        'PEPWELD' : {'val':4.5e-3, 'unit':'uM','moltype':'enz', 'kval':[
            KValue("kcat", "", 1e3)
        ]},
        'PK' : {'val':7.0e-2, 'unit':'uM','moltype':'enz', 'kval':[
            KValue("kcat", "", 1.7e3)
        ]},
}
mol

{'Gluc': {'val': 5000.0,
  'unit': 'uM',
  'moltype': 'met',
  'kval': [KValue(ktype='sat', ktarget='HK', kval=500.0)]},
 'F6P': {'val': 0.0,
  'unit': 'uM',
  'moltype': 'met',
  'kval': [KValue(ktype='sat', ktarget='PFKa', kval=1.0),
   KValue(ktype='sat', ktarget='PFKi', kval=10000000.0),
   KValue(ktype='alloI', ktarget='HK', kval=20.0)]},
 'F16BP': {'val': 0.0,
  'unit': 'uM',
  'moltype': 'met',
  'kval': [KValue(ktype='sat', ktarget='PEPWELD', kval=5000.0),
   KValue(ktype='alloA', ktarget='PK', kval=400.0)]},
 'PEP': {'val': 0.0,
  'unit': 'uM',
  'moltype': 'met',
  'kval': [KValue(ktype='sat', ktarget='PK', kval=35.0),
   KValue(ktype='sat', ktarget='PFKb', kval=0.05)]},
 'ADP': {'val': 560.0,
  'unit': 'uM',
  'moltype': 'met',
  'kval': [KValue(ktype='sat', ktarget='PK', kval=50.0),
   KValue(ktype='sat', ktarget='PFKb', kval=550.0)]},
 'ATP': {'val': 9600.0,
  'unit': 'uM',
  'moltype': 'met',
  'kval': [KValue(ktype='sat', ktarget='PFKa', kval=60.0),
   KValue(ktype='sat'

In [17]:
tfs = []
gain = []

for m in mol.keys():
    for kv in mol[m]['kval']:
        if kv.ktype == 'sat':
            tfs.append(saturation(mol[m]['val'], kv.kval))
        elif kv.ktype == 'alloI':
            tfs.append(alloInhib(mol[m]['val'], kv.kval))
        elif kv.ktype == 'alloA':
            tfs.append(alloAct(mol[m]['val'], kv.kval))
        elif kv.ktype == 'kcat':
            gain.append(kv.kval)
        else:
            if kv == None:
                pass
            else:
                print("uh oh, something went wrong")    


        if kv.ktype != 'kcat':
            print(f"mol: {m} ktype: {kv.ktype} target: {kv.ktarget} val: {tfs[-1]}")
        else:
            print(f"enz: {m} kcat: {kv.kval}")

tfs,gain

mol: Gluc ktype: sat target: HK val: 0.9090909090909091
mol: F6P ktype: sat target: PFKa val: 0.0
mol: F6P ktype: sat target: PFKi val: 0.0
mol: F6P ktype: alloI target: HK val: 1.0
mol: F16BP ktype: sat target: PEPWELD val: 0.0
mol: F16BP ktype: alloA target: PK val: 1.0
mol: PEP ktype: sat target: PK val: 0.0
mol: PEP ktype: sat target: PFKb val: 0.0
mol: ADP ktype: sat target: PK val: 0.9180327868852459
mol: ADP ktype: sat target: PFKb val: 0.5045045045045045
mol: ATP ktype: sat target: PFKa val: 0.9937888198757764
mol: ATP ktype: sat target: PFKi val: 0.9937888198757764
mol: ATP ktype: sat target: HK val: 0.3902439024390244
mol: ATP ktype: alloI target: PK val: 0.5555555555555556
enz: PFKa kcat: 100.0
enz: PFKi kcat: 100.0
enz: PFKb kcat: 0.0
enz: HK kcat: 4000.0
enz: PEPWELD kcat: 1000.0
enz: PK kcat: 1700.0


([0.9090909090909091,
  0.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.9180327868852459,
  0.5045045045045045,
  0.9937888198757764,
  0.9937888198757764,
  0.3902439024390244,
  0.5555555555555556],
 [100.0, 100.0, 0.0, 4000.0, 1000.0, 1700.0])

In [27]:
relatf = []

for m in mol.keys():
    for kv in mol[m]['kval']:
        if kv.ktype == 'sat':
            # jerryrig to prevent 0s in denom. This is just a metric and will only be used for debugging 
            if mol[m]['val'] == 0:
                mol[m]['val'] += 1e-16


            cv = np.exp( (np.log(mol[kv.ktarget]['val']) - np.log( (1 + kv.kval/mol[m]['val'] )) )   ) / mol[kv.ktarget]['val']
            relatf.append(cv)
            #print(cv)
        
        
        elif kv.ktype == 'alloI':
            cv = np.exp( (np.log(mol[kv.ktarget]['val']) - np.log( (1 + mol[m]['val']/kv.kval )) )   ) / mol[kv.ktarget]['val']
            relatf.append(cv)
        elif kv.ktype == 'alloA':
            cv = np.exp( (np.log(mol[kv.ktarget]['val']) + np.log( (1 + mol[m]['val']/kv.kval )) )   ) / mol[kv.ktarget]['val']
            relatf.append(cv)
        elif kv.ktype == 'kcat':
            gain.append(kv.kval)
        else:
            if kv == None:
                pass
            else:
                print("uh oh, something went wrong")    


        if kv.ktype != 'kcat':
            print(f"mol: {m} ktype: {kv.ktype} target: {kv.ktarget} val: {relatf[-1]}")
        else:
            print(f"enz: {m} kcat: {kv.kval}")


mol: Gluc ktype: sat target: HK val: 0.9090909090909095
mol: F6P ktype: sat target: PFKa val: 9.999999999999967e-17
mol: F6P ktype: sat target: PFKi val: 9.999999999999997e-24
mol: F6P ktype: alloI target: HK val: 1.0000000000000004
mol: F16BP ktype: sat target: PEPWELD val: 2.000000000000004e-20
mol: F16BP ktype: alloA target: PK val: 1.0
mol: PEP ktype: sat target: PK val: 2.857142857142853e-18
mol: PEP ktype: sat target: PFKb val: 1.9999999999999978e-15
mol: ADP ktype: sat target: PK val: 0.9180327868852457
mol: ADP ktype: sat target: PFKb val: 0.5045045045045046
mol: ATP ktype: sat target: PFKa val: 0.9937888198757762
mol: ATP ktype: sat target: PFKi val: 0.9937888198757762
mol: ATP ktype: sat target: HK val: 0.3902439024390246
mol: ATP ktype: alloI target: PK val: 0.5555555555555556
enz: PFKa kcat: 100.0
enz: PFKi kcat: 100.0
enz: PFKb kcat: 0.0
enz: HK kcat: 4000.0
enz: PEPWELD kcat: 1000.0
enz: PK kcat: 1700.0


In [26]:



class Molecule:
    """
    Molecule Data Class structure

    Attributes:
    -----------
        name : `str`, default = ""
            The name of the molecule
        val : `float`, default = 0.0
            The quantity of the molecule
        unit : `str`, default = "count"
            The unit for the quantity of the molecule
        molType: `str`, default = "met"
            The type of molecule.  Currently accepted values are "met" and "enz"
        kval : `dict`, default = {}
            The kvalues associated with the molecule. 
            If the molecule is an enzyme "enz": kvalues indicate kcat and should be {"kval": kval}
            If the Molecule is a metabolite "met", kvalues indicate either KM or Kreg
            

    Notes:
    ------
    Potential Future attributes:
    pos:
    container:
    """
    def __init__(self, name:str = "", val:float = 0.0, unit:str = "count", moltype:str = "met"):
        self.name = name
        self.val = val
        self.unit = unit
        self.moltype = moltype
        self.kval = [[], []]

    def getName(self):
        return self.name

    def addEnzKval(self, enzKval):
        self.kval[1].append(enzKval)

    def __print__(self):
        print(f"name: {self.name}")
        print(f"val: {self.val}")
        print(f"unit: {self.unit}")
        print(f"moltype: {self.moltype}")
        print(f"kval: {self.kval}")

# Breaking this down into molecules, stochiometry, and reactions
Molecules = np.array([Molecule(key, DICT_T0_CONC[key][0], 'uM', DICT_T0_CONC[key][1]) for key in DICT_T0_CONC.keys()])
print(Molecules)
print([Molecules[i].__print__() for i in range(len(Molecules))])
print(Molecules[0].name)
print(Molecules[0].val)
m = Molecules

name: Gluc
val: 5000
unit: uM
moltype: met
kval: [[], []]
name: F6P
val: 0
unit: uM
moltype: met
kval: [[], []]
name: F16BP
val: 0
unit: uM
moltype: met
kval: [[], []]
name: PEP
val: 0
unit: uM
moltype: met
kval: [[], []]
name: ADP
val: 560
unit: uM
moltype: met
kval: [[], []]
name: ATP
val: 9600
unit: uM
moltype: met
kval: [[], []]
name: PYR
val: 0
unit: uM
moltype: met
kval: [[], []]
name: PFKa
val: 0.001
unit: uM
moltype: enz
kval: [[], []]
name: PFKi
val: 0.001
unit: uM
moltype: enz
kval: [[], []]
name: PFKb
val: 0.005
unit: uM
moltype: enz
kval: [[], []]
name: HK
val: 0.007
unit: uM
moltype: enz
kval: [[], []]
name: PEPWELD
val: 0.0045
unit: uM
moltype: enz
kval: [[], []]
name: PK
val: 0.07
unit: uM
moltype: enz
kval: [[], []]
[None, None, None, None, None, None, None, None, None, None, None, None, None]
Gluc
5000


In [27]:
# horribly inefficient -- but it works

for mol in range(len(m)):
    #print(enz, mol)
    if m[mol].name in DICT_KCAT.keys():
        #print(m[mol].name, mol, DICT_KCAT[ m[mol].getName() ])
        #print(m[mol].kval)
        m[mol].kval[1] = [DICT_KCAT[ m[mol].getName() ]]
        #print(m[mol].kval)

    if m[mol].name in DICT_ENZ_SUBSTRATE.keys():
        for effector in DICT_ENZ_SUBSTRATE[m[mol].getName()].keys():
            if effector == 'prod':
                pass
            else:
                for eff in DICT_ENZ_SUBSTRATE[m[mol].getName()][effector]:
                    # Go back through and append the correct kvalue to the correct molecule
                    for omol in range(len(m)):
                        if m[omol].name == eff[0]:
                            if eff[1] == 'sat':
                                m[omol].kval[0].insert(0, (m[mol].getName(), eff[0], eff[1], DICT_KM[f"{m[mol].getName()}_{eff[0]}"])   )
                            elif eff[1] == 'alloI':
                                m[omol].kval[0].append(( m[mol].getName(), eff[0], eff[1], DICT_K_REGULATORY[f"Ki_{m[mol].getName()}_{eff[0]}"])  )
                            elif eff[1] == 'alloA':
                                m[omol].kval[0].append(( m[mol].getName(), eff[0], eff[1], DICT_K_REGULATORY[f"Ka_{m[mol].getName()}_{eff[0]}"])  )
                            

printBlockMessage('DOONE')
print([m[i].__print__() for i in range(len(m))])

############################################
DOONE
############################################
name: Gluc
val: 5000
unit: uM
moltype: met
kval: [[('HK', 'Gluc', 'sat', 500.0)], []]
name: F6P
val: 0
unit: uM
moltype: met
kval: [[('PFKi', 'F6P', 'sat', 10000000.0), ('PFKa', 'F6P', 'sat', 1.0)], []]
name: F16BP
val: 0
unit: uM
moltype: met
kval: [[('PEPWELD', 'F16BP', 'sat', 5000.0)], []]
name: PEP
val: 0
unit: uM
moltype: met
kval: [[('PK', 'PEP', 'sat', 35.0), ('PFKb', 'PEP', 'sat', 0.05)], []]
name: ADP
val: 560
unit: uM
moltype: met
kval: [[('PK', 'ADP', 'sat', 50.0), ('PFKb', 'ADP', 'sat', 550.0)], []]
name: ATP
val: 9600
unit: uM
moltype: met
kval: [[('HK', 'ATP', 'sat', 15000.0), ('PFKi', 'ATP', 'sat', 60), ('PFKa', 'ATP', 'sat', 60)], []]
name: PYR
val: 0
unit: uM
moltype: met
kval: [[], []]
name: PFKa
val: 0.001
unit: uM
moltype: enz
kval: [[], [100.0]]
name: PFKi
val: 0.001
unit: uM
moltype: enz
kval: [[], [100.0]]
name: PFKb
val: 0.005
unit: uM
moltype: enz
kval: [[], []]
name

In [8]:

def TF(enz:Molecule, targets:list):
    # Find the relevant transfer functions
    tfs = [(targets[i], targets[i].kval["met"][j], 0) for i in range(len(targets)) for j in range(len(targets[i].kval["met"])) if targets[i].kval["met"][i]['ktarget'] == enz.name]

    # for each of those:
    for tf in range(len(tfs)):
        if tfs[tf][1]['ktype'] == 'sat':
            tfs[tf][2] = saturation(tfs[tf][0].val, tfs[tf][1]['kval'])
    
    return tfs





In [ ]:
class EnzymeTransferFunction:
    def __init__(self, enzyme:Molecule, targets:list):
        self.vmax = enzyme.val * enzyme.kval[1]
        self.logEnz = np.log(enzyme.val)
        self.saturation = self.getSaturation()
        self.allostery = self.getAllostery()

    def getChildTransferFunctions(self):
        # for each substrate in the substrate list find the values which have the current enzyme as a target
        # Return these as a list
        # Then for each in the stored list, calculate the log Sat factor
        
        pass

    def getAllostery(self):
        # for each substrate in the substrate list find the values which have the current enzyme as a target
        # Return these as a list
        # Then for each in the stored list, calculate the log Sat factor
        pass



In [3]:
def hexokinase(self):
        """ Gluc + ATP --> F6P + ADP; Catalyzed by HK; Allosterically inhibited by Pyruvate and F6P"""
        vmax = self.dictKcat['HK'] * self.molecules['HK'] 
        sat = saturation(self.molecules['Gluc'], self.dictKM['HK_Gluc']) * saturation(self.molecules['ATP'], self.dictKM['HK_ATP'])
        allo =  alloInhib(self.molecules['F6P'], self.dictKreg['Ki_F6P_HK'])
        return vmax * sat * allo

r = ['HK', 'Gluc', 'ATP', 'F6P', 'ADP']
s = [1,-1,-1,0,0] --> [1,0,0,1,1]

### So it gets kinda messy: (And this doesn't even include the ATP/ADP)
['HK', 'Gluc', 'F6P', 'HK*Gluc', HK*F6P(c), HK*F6P(i), HK*Gluc*F6P(i), HK*F6P(c)*F6P(i)]
HK + Gluc <--> HK*Gluc
[-1,-1,0,1,0,0,0,0]
[1,1,0,-1,0,0,0,0]

HK + F6P <--> HK*F6P(i)
[-1,-1,0,0,0,1,0,0]
[1,1,0,0,0,-1,0,0]

HK*Gluc + F6P <--> HK*Gluc*F6P(i)
[0,-1,0,-1,0,0,1,0]
[0,1,0,1,0,0,-1,0]

HK*Gluc --> HK*F6P(c)
[0,0,0,-1,1,0,0,0]

HK*Gluc*F6P(i) --> HK*F6P(c)*F6P(i)
[0,0,0,0,0,0,-1,1]
HK*F6P(c) --> HK + F6P
[1,0,1,0,-1,0,0,0]
HK*F6P(c)*F6P(i) --> HK*F6P(i) + F6P
[0,0,1,0,0,1,0,-1]
HK*F6P(c)*F6P(i) --> HK*F6P(c) + F6P
[0,0,1,0,1,0,0,-1]

In [15]:
#['HK', 'Gluc', 'F6P', 'HK*Gluc', HK*F6P(c), HK*F6P(i), HK*Gluc*F6P(i), HK*F6P(c)*F6P(i)]
glyc = [
    #HK + Gluc <--> HK*Gluc
    [-1,-1,0,1,0,0,0,0],
    [1,1,0,-1,0,0,0,0],
    #HK + F6P <--> HK*F6P(i)
    [-1,-1,0,0,0,1,0,0],
    [1,1,0,0,0,-1,0,0],
    #HK*Gluc + F6P <--> HK*Gluc*F6P(i)
    [0,-1,0,-1,0,0,1,0],
    [0,1,0,1,0,0,-1,0],
    #HK*Gluc --> HK*F6P(c)
    [0,0,0,-1,1,0,0,0],
    #HK*Gluc*F6P(i) --> HK*F6P(c)*F6P(i)
    [0,0,0,0,0,0,-1,1],
    #HK*F6P(c) --> HK + F6P
    [1,0,1,0,-1,0,0,0],
    #HK*F6P(c)*F6P(i) --> HK*F6P(i) + F6P
    [0,0,1,0,0,1,0,-1],
    #HK*F6P(c)*F6P(i) --> HK*F6P(c) + F6P
    [0,0,1,0,1,0,0,-1],
]
glyc

[[-1, -1, 0, 1, 0, 0, 0, 0],
 [1, 1, 0, -1, 0, 0, 0, 0],
 [-1, -1, 0, 0, 0, 1, 0, 0],
 [1, 1, 0, 0, 0, -1, 0, 0],
 [0, -1, 0, -1, 0, 0, 1, 0],
 [0, 1, 0, 1, 0, 0, -1, 0],
 [0, 0, 0, -1, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, -1, 1],
 [1, 0, 1, 0, -1, 0, 0, 0],
 [0, 0, 1, 0, 0, 1, 0, -1],
 [0, 0, 1, 0, 1, 0, 0, -1]]

glycsym = [
    [-a,-a,0,a,0,0,0,0],
    [-c,-c,0,0,0,c,0,0],
    [d,d,0,0,0,-d,0,0], 
    [0,-e,0,-e,0,0,e,0],
    [0,f,0,f,0,0,-f,0], 
    [0,0,0,-g,g,0,0,0], 
    [0,0,0,0,0,0,-h,h], 
    [i,0,i,0,-i,0,0,0], 
    [0,0,j,0,0,j,0,-j], 
    [0,0,k,0,k,0,0,-k], 
]

In [ ]:
#['HK', 'Gluc', 'F6P', 'HK*Gluc', HK*F6P(c), HK*F6P(i), HK*Gluc*F6P(i), HK*F6P(c)*F6P(i)]
glycsym = [
    [-a,-a,0,a,0,0,0,0],#HK + Gluc --> HK*Gluc
    [b,b,0,-b,0,0,0,0], #HK + Gluc <-- HK*Gluc
    [-c,-c,0,0,0,c,0,0],#HK + F6P --> HK*F6P(i)
    [d,d,0,0,0,-d,0,0], #HK + F6P <-- HK*F6P(i)
    [0,-e,0,-e,0,0,e,0],#HK*Gluc + F6P --> HK*Gluc*F6P(i)
    [0,f,0,f,0,0,-f,0], #HK*Gluc + F6P <-- HK*Gluc*F6P(i)
    [0,0,0,-g,g,0,0,0], #HK*Gluc --> HK*F6P(c)
    [0,0,0,0,0,0,-h,h], #HK*Gluc*F6P(i) --> HK*F6P(c)*F6P(i)
    [i,0,i,0,-i,0,0,0], #HK*F6P(c) --> HK + F6P
    [0,0,j,0,0,j,0,-j], #HK*F6P(c)*F6P(i) --> HK*F6P(i) + F6P
    [0,0,k,0,k,0,0,-k], #HK*F6P(c)*F6P(i) --> HK*F6P(c) + F6P
]
glycsym

In [12]:
class EnzymeReaction:
    """
    Reaction Data Class structure (Enzyme transfer Function)

    Attributes:
    -----------
        name : `str`, default = ""
            The name of the reaction
        subs : `list(str)`, default = ['']
            List of molecule names on the LHS of the reaction 
        prod : `list(str)`, default = ['']
            List of molecule names on the RHS of the reaction
        qSubs : `list(int)`, default = [1]
            The quantity of each substrate (stochiometry) consumed in 1 reaction
        qProd : `list(int)`, default = [1]
            The quantity of each product (stochiometry) produced in 1 reaction

    Notes:
    ------
    If a molecule is both a `subs` and a `prod` then it is considered a catalyst

    """
    name: str = ""
    subs: dict = [""]
    prod: list = [""]
    qSubs: list = [1]
    qProd: list = [1]
    
    

In [16]:
class System:
    """
    System Class

    Attributes:
    -----------
        ss : `list(Molecule)` 
            The list of molecules comprising the system state space(ss).
        rxn :  `list(Reaction)`
            The list of reactions between states

    Notes:
    ------

    """
    def __init__(self, ss = [Molecule()], rxn = [Reaction()]):
        self.ss = ss
        self.rxn = rxn    



5000

In [29]:
t = {
    "a":{'1':1, '2':2, '3':3},
    "b":{'1':1, '2':2, '3':3},
    "c":{'1':1, '2':2, '3':3},
}
t

{'a': {'1': 1, '2': 2, '3': 3},
 'b': {'1': 1, '2': 2, '3': 3},
 'c': {'1': 1, '2': 2, '3': 3}}

In [30]:
t['a']['1'] = 1234
t

{'a': {'1': 1234, '2': 2, '3': 3},
 'b': {'1': 1, '2': 2, '3': 3},
 'c': {'1': 1, '2': 2, '3': 3}}

In [13]:
conc = [10**(i/4) for i in range(-28, 20)]
#X, Y, Z = conc, conc, conc
# dict_K= {
# 'HK_Gluc' : 500.0,
# 'HK_ATP' : 15000.0,
# 'Ki_F6P_HK' : 20.0,}

In [15]:
def hexokinase(Gluc, ATP, F6P, Gluc_KM, ATP_KM, F6P_Ki):
        """ Gluc + ATP --> F6P + ADP; Catalyzed by HK; Allosterically inhibited by Pyruvate and F6P"""
        sat = saturation(Gluc, Gluc_KM) * saturation(ATP, ATP_KM)
        allo = alloInhib(F6P, F6P_Ki)
        return sat * allo

def HK_sat_Gluc(Gluc, Gluc_KM=500.0):
    return saturation(Gluc, Gluc_KM)
def HK_sat_ATP(ATP, ATP_KM= 15000.0):
    return saturation(ATP, ATP_KM)
def HK_alloI_F6P(F6P, F6P_Ki=20.0):
    return alloInhib(F6P, F6P_Ki)


In [16]:
X, Y, Z = [HK_sat_Gluc(i) for i in conc], [HK_sat_ATP(i) for i in conc], [HK_alloI_F6P(i) for i in conc]

In [55]:

def pplot_ioEnz(
    #enzName,
    #subsName,
    start = -9,
    stop = 5,
    stepsPerLog10 = 10
    ):
    """# I like the turbo color scale (continuous)"""
    # Set up the scale
    conc = [10**(i/stepsPerLog10) for i in range(start*stepsPerLog10,stop*stepsPerLog10)]

    X, Y, Z = conc, conc, [[HK_sat_Gluc(i)*HK_sat_ATP(j)*100 for j in conc] for i in conc]
    #print(Z)
    # # Basically I need to pass in an enzyme, then 
    # if enzName == "HK":

    fig = go.Figure(
        data = [go.Surface(x = X, y = Y, z = Z, color= Z, colorscale="turbo")],
    )
    # fig.update_traces(contours_z=dict(show=True, usecolormap=True,
    #                               highlightcolor="limegreen", project_z=True))

    # fig.update_xaxes(type = 'log')
    # fig.update_yaxes(type = 'log')                              

    return fig
    

In [57]:

# fig = pplot_ioEnz()
# fig.show()

In [60]:
pplot_ioEnz().show()